## 使用字符RNN生成莎士比亚文本

在2015年的一篇著名博客文章 "https://karpathy.github.io/2015/05/21/rnn-effectiveness/" 中，Andrej Karpathy展示了如何训练循环神经网络来预测句子中的下一个字符。这个char-RNN可以用来生成小说文本，每次一个字符。

In [1]:
# tf.keras.utils.get_file()函数下载莎士比亚的所有作品
import tensorflow as tf

shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [2]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [3]:
# 使用tf.keras.layers.TextVectorization层对此文本进行编码。我们将设置split="character"，基于字符进行编码，而不是默认的基于单词进行编码，并使用standardize="lower"将文本转换为小写（这将简化任务）：
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")

text_vec_layer.adapt([shakespeare_text])


encoded = text_vec_layer([shakespeare_text])[0]
encoded


<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([21,  7, 10, ..., 22, 28, 12], dtype=int64)>

In [4]:
text_vec_layer.get_vocabulary()

['',
 '[UNK]',
 ' ',
 'e',
 't',
 'o',
 'a',
 'i',
 'h',
 's',
 'r',
 'n',
 '\n',
 'l',
 'd',
 'u',
 'm',
 'y',
 'w',
 ',',
 'c',
 'f',
 'g',
 'b',
 'p',
 ':',
 'k',
 'v',
 '.',
 "'",
 ';',
 '?',
 '!',
 '-',
 'j',
 'q',
 'x',
 'z',
 '3',
 '&',
 '$']

In [5]:
# 每个字符现在都映射到一个整数，从2开始。TextVectorization层将值0保留为填充词元(token)，并将值1保留为未知字符。
# 现在不需要这两个词元，因此从字符ID数中减去2，并计算不同字符的总数和总字符数：
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

print(n_tokens)

39


将这个非常长的序列转换成一个窗口数据集，然后使用它来训练一个序列到序列的循环神经网络。目标序列将类似于输入序列，但是会向“未来”移动一个时间步。例如，数据集中的样本可能是一个由表示文本“to be or not to b”（不包括最后一个“e”）的字符ID组成的序列，相应的目标序列是一个由表示文本“o be or not to be”（包括最后一个“e”，但不包括开头的“t”）的字符ID组成的序列

In [6]:
# 将字符ID序列转换成输入/目标窗口对的数据集
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)

    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))


    if shuffle:
        ds = ds.shuffle(buffer_size=100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

- 它将序列作为输入（即编码文本），并创建一个包含所需长度的所有窗口的数据集。
- 它将长度增加1，因为我们需要将下一个字符放在目标序列中。
- 然后，它对窗口进行乱序处理（可选），将它们分批，将它们拆分为输入/输出对，并激活预取功能。

长度为11的窗口和批量大小3。每个窗口的起始索引都显示在旁边：

![准备乱序窗口的数据集](./images/RNN/p7.png)

In [7]:
# 大约90%的文本进行训练，5%用于验证，5%用于测试
length = 100  # length决定循环神经网络能学习的最长模式
tf.random.set_seed(42)

train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000: 1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

In [8]:
for data in valid_set.take(1):
    print(data)

(<tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[ 5,  7,  0, ...,  6,  1,  0],
       [ 7,  0,  8, ...,  1,  0, 18],
       [ 0,  8,  1, ...,  0, 18,  6],
       ...,
       [ 4,  2,  0, ...,  3, 26, 10],
       [ 2,  0,  7, ..., 26, 10, 10],
       [ 0,  7,  6, ..., 10, 10,  2]], dtype=int64)>, <tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[ 7,  0,  8, ...,  1,  0, 18],
       [ 0,  8,  1, ...,  0, 18,  6],
       [ 8,  1,  4, ..., 18,  6,  3],
       ...,
       [ 2,  0,  7, ..., 26, 10, 10],
       [ 0,  7,  6, ..., 10, 10,  2],
       [ 7,  6,  1, ..., 10,  2,  8]], dtype=int64)>)


### 构建和训练char-RNN模型

- 使用Embedding层作为第一层，以编码字符ID。Embedding层的输入维数是不同字符ID的数量，输出维数是可以调整的超参数——现在将其设置为16。Embedding层的输入将是形状为［批量大小，窗口长度］的二维张量，Embedding层的输出将是形状为［批量大小，窗口长度，嵌入大小］的三维张量。
- 为输出层使用Dense层：它必须具有39个单元(n_tokens)，因为文本中有39个不同的字符，希望在每个时间步输出每个可能字符的概率。每个时间步39个输出概率的总和应该为1，因此对Dense层的输出应用softmax激活函数。
- 最后，使用"sparse_categorical_crossentropy"损失和Nadam优化器编译此模型，并使用ModelCheckpoint回调函数在训练过程中保存最佳模型（以验证精度为标准）进行多个轮次的训练。

In [10]:
# 不用简单RNN,用带有长短记忆的GRU
#
# [[ 5,  7,  0, ...,  6,  1,  0]]  1 * 100
# [[ v5  v7  v0  .... v6  v1  v0]] 1 * 100 * 16

# n_tokens:39
# embedding： 39 * n 列的矩阵：  5行：  [w0, w1, ... wn-1]

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model.keras", monitor="val_accuracy", save_best_only=True
)
history = model.fit(train_set, validation_data=valid_set, epochs=1, callbacks=[model_ckpt])

  31247/Unknown 991s 31ms/step - accuracy: 0.5455 - loss: 1.5034

C:\Users\31752\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


31247/31247 ━━━━━━━━━━━━━━━━━━━━ 1008s 32ms/step - accuracy: 0.5745 - loss: 1.3875 - val_accuracy: 0.5335 - val_loss: 1.6007


In [22]:
 # 训练后搭建最终模型
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X-2), # 不使用填充词元(0)  和 未知词元 (1)
    model
])

In [24]:
y_proba = shakespeare_model.predict(tf.constant(["To be or not to b"]))[0, -1]
y_proba
y_pred = tf.argmax(y_proba)  # 选择概率最高的字母ID

print(text_vec_layer.get_vocabulary())

text_vec_layer.get_vocabulary()[y_pred + 2]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step
['', '[UNK]', ' ', 'e', 't', 'o', 'a', 'i', 'h', 's', 'r', 'n', '\n', 'l', 'd', 'u', 'm', 'y', 'w', ',', 'c', 'f', 'g', 'b', 'p', ':', 'k', 'v', '.', "'", ';', '?', '!', '-', 'j', 'q', 'x', 'z', '3', '&', '$']


'e'

### 生成莎士比亚文本

要使用char-RNN模型生成新文本，可以将一些文本输入模型，让模型预测最有可能的下一个字母，将其添加到文本的末尾，然后将扩展后的文本输入模型来猜测下一个字母，以此类推。这叫作贪婪解码。但是在实践中，这往往导致相同的单词一遍又一遍地重复。

相反，可以使用TensorFlow的tf.random.categorical()函数随机采样下一个字符，采样概率等于估计概率。这将生成更多样化和有趣的文本。categorical()函数在给定类别对数概率(logits)的情况下，对随机类别指数进行采样。

In [25]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])
tf.random.set_seed(42)

n = 2000
tf.random.categorical(log_probas, num_samples=n)



<tf.Tensor: shape=(1, 2000), dtype=int64, numpy=array([[0, 1, 0, ..., 1, 0, 2]], dtype=int64)>

为了更好地控制生成文本的多样性，可以将logits（对数概率）除以一个称为“温度”的数字，这个数字可以根据需求进行调整。“温度”接近零将更偏向于高概率字符，而较高“温度”则会使所有字符获得相同的概率。通常在生成相对严谨和精确的文本（例如数学公式）时，较低的“温度”更为适用，而在生成更多样化且有创意的文本时，则适合用较高的“温度”。

In [34]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict(tf.constant([text]))[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [35]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [36]:
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature=1))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━

In [37]:
print(extend_text("To be or not to be", temperature=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━

In [29]:
print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 16ms/step
To be or not to bef ,mt'&ozfpady-$
wh!nse?pws3ert--vgerdjw?c-y-ewxnj


### 模拟生成莎士比亚文本的流程 生成名字

In [1]:
with open("datasets/dino.txt", "r") as f:
    dino_names = f.read()
    dino_names = dino_names.lower()

In [2]:
import tensorflow as tf
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")

text_vec_layer.adapt([dino_names])
encoded = text_vec_layer([dino_names])[0]
encoded

<tf.Tensor: shape=(19909,), dtype=int64, numpy=array([ 2,  2, 15, ...,  4,  4, 12], dtype=int64)>

In [3]:
text_vec_layer.get_vocabulary()

['',
 '[UNK]',
 'a',
 's',
 'u',
 'o',
 'r',
 '\n',
 'n',
 'i',
 'e',
 't',
 'l',
 'p',
 'h',
 'c',
 'g',
 'd',
 'm',
 'y',
 'b',
 'k',
 'v',
 'x',
 'z',
 'j',
 'w',
 'f',
 'q']

In [4]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)
n_tokens

27

In [5]:
end_of_name_encode = text_vec_layer(["\n"])[0,0].numpy() - 2
end_of_name_encode

5

In [6]:
# 把 Tensor 转成 numpy 数组
encoded_np = encoded.numpy()

# 存放 (X, Y) 对
names_X = []  # [第一个名字不包括换行符， 第二个名字不包括换行符， ...  最后一个名字不包括换行符]
names_Y = []  # [第一个名字有换行符，但无第一个字符， .....    , ...  最后一个名字有换行符，但无第一个字符]

# 临时缓冲区
current_name = []

for token in encoded_np:
    current_name.append(token)
    if token == end_of_name_encode:
        if len(current_name) > 1:  # 至少两个字符才能形成 X/Y
            X = current_name[:-1]
            Y = current_name[1:]
            names_X.append(X)
            names_Y.append(Y)
        current_name = []


current_name.append(end_of_name_encode)
if len(current_name) > 1:
    X = current_name[:-1]
    Y = current_name[1:]
    names_X.append(X)
    names_Y.append(Y)
    current_name = []

names_X[:5], names_Y[:5]
# 转成 TensorFlow 张量
X_dataset = tf.ragged.constant(names_X)
X_dataset
Y_dataset = tf.ragged.constant(names_Y)
Y_dataset

train_set = tf.data.Dataset.from_tensor_slices((X_dataset, Y_dataset)).shuffle(1000).batch(8)
print("共提取名字数:", len(names_X))


共提取名字数: 1536


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

# history = model.fit(X_dataset, Y_dataset, batch_size=1, epochs=10)
model.fit(train_set, epochs=20)

Epoch 1/20
Instructions for updating:
Use fn_output_signature instead
192/192 [==============================] - 3s 5ms/step - loss: 2.4201 - accuracy: 0.3095
Epoch 2/20
192/192 [==============================] - 1s 5ms/step - loss: 1.9634 - accuracy: 0.4251
Epoch 3/20
192/192 [==============================] - 1s 5ms/step - loss: 1.8499 - accuracy: 0.4477
Epoch 4/20
192/192 [==============================] - 1s 5ms/step - loss: 1.7907 - accuracy: 0.4607
Epoch 5/20
192/192 [==============================] - 1s 5ms/step - loss: 1.7383 - accuracy: 0.4744
Epoch 6/20
192/192 [==============================] - 1s 5ms/step - loss: 1.7000 - accuracy: 0.4894
Epoch 7/20
192/192 [==============================] - 1s 5ms/step - loss: 1.6651 - accuracy: 0.4938
Epoch 8/20
192/192 [==============================] - 1s 5ms/step - loss: 1.6380 - accuracy: 0.5030
Epoch 9/20
192/192 [==============================] - 1s 5ms/step - loss: 1.6126 - accuracy: 0.5140
Epoch 10/20
192/192 [====================

In [8]:
dino_name_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X -2),
    model
])

In [9]:
def my_next_char(model, text, temperature=1):
    y_proba = model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [10]:
def my_extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        char_gen_next = my_next_char(dino_name_model, text, temperature)
        if char_gen_next == "\n":
            break
        text += my_next_char(dino_name_model, text, temperature)
    return text

In [13]:
my_extend_text("xx")

1/1 [==============================] - 0s 33ms/step


'xxaveisaurus'